In [ ]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
import cv2
import numpy as np
import math
import random
import bisect
import operator
import matplotlib.pyplot as plt
import copy
import sys
import os
import shutil
import datetime
import time

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)


In [ ]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [ ]:
def func(latents,text_embeddings):
    
    num_inference_steps = 50            # Number of denoising steps

    guidance_scale = 7.5                # Scale for classifier-free guidance

    #############################################

    uncond_input = tokenizer(
        [""] * batch_size, padding="max_length", return_tensors="pt"
    )
    with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

    ###############################################

    embeddings = torch.cat([uncond_embeddings, text_embeddings])

    #############################################

    scheduler.set_timesteps(num_inference_steps)

    ##########################################

    latents_1 = latents * scheduler.init_noise_sigma

    #############################################



    for t in tqdm(scheduler.timesteps):
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input_a = torch.cat([latents_1] * 2)

      latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

    ###############################################
    # scale and decode the image latents with vae
    latents_3 = 1 / 0.18215 * latents_1

    with torch.no_grad():
      image = vae.decode(latents_3).sample


    ##########################################

    image_1 = (image / 2 + 0.5).clamp(0, 1)
    image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image_2 * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    result_image = pil_images[0]
    
    return result_image

In [ ]:
#gaの関数
population = 9 #個体数
X = 3
Y = 3
mutation = 0 #突然変異の個体数
# population = 3 #個体数
# X = 1
# Y = 3
mutation = 2 #突然変異の個体数

if X*Y!=population or population>9:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)
    
gene_length = 64
elite = 0 #エリートの数
initializa_txt_num = 70 #初期化個体においてtxtのベクトルをどれくらい元から変異させるか
batch_size = 1
height = 512                        # default height of Stable Diffusion
width = 512                        # default width of Stable Diffusion
image_mutation_rate = 400
text_mutation_rate = 500

if elite+mutation>population:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)

#seed値を複数個用意
def initialize_gene(text_embeddings):
    arr = []
    for i in range(population):
        embeddings = text_embeddings.clone()
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            text_embeddings[0][a][b] = np.random.randn()
        arr.append([latents,embeddings])    
    return arr


In [ ]:
def save_image(diffusion_images,epoch,folder):
    os.mkdir(folder+'/epoch'+str(epoch))
    for i in range (len(diffusion_images)):
        diffusion_images[i].save(folder+"/epoch"+str(epoch)+"/"+str(i)+".png")
    
    
def evolve(selected,genes):
    global evolve_explanation
    evolve_explanation = ""
    new_genes = []
    
    #エリート戦略 #選ばれた画像は次世代に残す(?)
#     for i in range(population):
#         if selected[i]==1:
#             adding_gene = copy.deepcopy(genes[i])
#             evolve_explanation += str(len(new_genes))+". elite"+str(i)+"\n"
#             new_genes.append(adding_gene)
    
    #mutation, cross overで使う配列の準備
    index = []
    num = 0
    for i in range(population):
        num += 1+selected[i]*adapt
        index.append(num)
    index.append(num)
    
    #mutation
    for i in range(mutation):
        p1 = random.randrange(num)
        g1 = bisect.bisect(index,p1)
        evolve_explanation += str(len(new_genes))+". mutation "+str(g1)+"\n"
        new_gene1 = copy.deepcopy(genes[g1])
        # 画像の元
        for j in range(4):
            for k in range(64):
                for l in range(64):
                    mutation_flag = random.randrange(image_mutation_rate)
                    if mutation_flag==0:
                        new_gene1[0][0][j][k][l] = np.random.randn()
        #textの元
        for j in range(77):
            for k in range(768):
                mutation_flag = random.randrange(text_mutation_rate)
                if mutation_flag==0:# cross over
    for i in range(population-len(new_genes)):
        p1 = random.randrange(num)
        p2 = random.randrange(num)
        g1 = bisect.bisect(index,p1)
        g2 = bisect.bisect(index,p2)
        new_gene = copy.deepcopy(genes[g1])
        new_gene1 = copy.deepcopy(genes[g2])
        evolve_explanation += str(len(new_genes))+". crossover "+str(g1)+" and "+str(g2)+"\n"
        cross = random.randrange(gene_length)
        for j in range(4):
            for k in range(64):
                for l in range(64):
                    a = random.randrange(2)
                    if a==0:
                        new_gene[0][0][j][k][l] = new_gene1[0][0][j][k][l]
        for j in range(77):
            for k in range(768):
                new_gene[1][0][j][k] = new_gene1[1][0][j][k]
        new_genes.append(new_gene)
                    
    if len(new_genes)!=population:
        print('Error: evolve error', file=sys.stderr)
        sys.exit(1)
    return new_genes
                    new_gene1[1][0][j][k] = np.random.randn()
        new_genes.append(new_gene1)
    # cross over
    for i in range(population-len(new_genes)):
        p1 = random.randrange(num)
        p2 = random.randrange(num)
        g1 = bisect.bisect(index,p1)
        g2 = bisect.bisect(index,p2)
        new_gene = copy.deepcopy(genes[g1])
        new_gene1 = copy.deepcopy(genes[g2])
        evolve_explanation += str(len(new_genes))+". crossover "+str(g1)+" and "+str(g2)+"\n"
        cross = random.randrange(gene_length)
        for j in range(4):
            for k in range(64):
                for l in range(64):
                    a = random.randrange(2)
                    if a==0:
                        new_gene[0][0][j][k][l] = new_gene1[0][0][j][k][l]
        for j in range(77):
            for k in range(768):
                new_gene[1][0][j][k] = new_gene1[1][0][j][k]
        new_genes.append(new_gene)
                    
    if len(new_genes)!=population:
        print('Error: evolve error', file=sys.stderr)
        sys.exit(1)
    return new_genes

In [ ]:
##########################################################
# tkinterの参考にしたサイト
# https://teratail.com/questions/202187
# buttonに関するサイト
# https://torimakujoukyou.com/python-tkinter-for-button/
# https://qiita.com/igor-bond16/items/39c8b75d844f30cea197
# 画像の切り替え
# https://joytas.net/programming/python/tkinter-img
# 画面の切り替え
# https://office54.net/python/tkinter/screen-change-tkraise
##########################################################

#ライブラリのインポート
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import random
import PIL
from PIL import Image,ImageTk
##############################################
#parameter
image_padding = 300
imgs = [] # Imageを保持するリストを追加
iec_buttons = []
iec_selected = []
iec_selected_image_id = -1
conv_selected_image_id = -1
canvases = []
conv_canvases = []
labels = []
for i in range(X*Y):
    iec_selected.append(0)
iec_epoch = 0 #iecでの試行回数
conv_epoch = 0 #従来手法での試行回数
adapt = 5 #iecにおいてuserに選ばれた遺伝子は選ばれていない遺伝子よりadapt倍選ばれやすい
evolve_explanation = "evolution explanation"
#diffusionの処理
iec_target_theme = "シンデレラ"
sample_image_path = "sea_image.png" #iecシステムの使い方を説明するときのサンプル画像
sample_image_prompt = "an illustration,　in the fantastic tropical sea, many fish, dazzling light coming from above, rocky shore"
sample_image_folder_path = "sample_image"
conv_target_theme = "かぐや姫"
prompt = []
iec_experiment_done = False #iecシステムの実験が終わったか
conv_experiment_done = False #stable diffusionシステムの実験が終わったか
current_system_is_practice1 = False
current_system_is_practice2 = False
current_system_is_live = False
practice_generation_limit = 3#練習時の試行回数の上限
live_generation_limit = 10#本番時の試行回数の上限
##############################################
def set_target_title(type,screen):
    if type=="iec":
        target_label = tk.Label(screen,text="theme", fg="white", foreground='blue',font=("Lucida Console","15"))
        target_label.place(x=150, y=70)
        target_theme_label = tk.Label(screen,text=iec_target_theme, foreground='black',font=("Lucida Console","15"))
        target_theme_label.place(x=150, y=100)
    elif type=="conv":
        target_label = tk.Label(screen,text="theme", fg="white", foreground='blue',font=("Lucida Console","15"))
        target_label.place(x=150, y=70)
        target_theme_label = tk.Label(screen,text=conv_target_theme, foreground='black',font=("Lucida Console","15"))
        target_theme_label.place(x=150, y=100)
    else:
        print('Error: target image invalid input', file=sys.stderr)
        sys.exit(1)
def showing_progress_screen_and_generate_images(type,screen):
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "progress_screen_is_displayed.", file=f)
    #プログレス画面
    progress_screen = tk.Frame(screen,bg='#505050',width=1500,height=900)
    progress_screen.lift()
    progress_screen.pack(fill=tk.BOTH)
    #プログレスバー
    #プログレスバーの初期設定
    pbval = tk.IntVar(value=0)
    progressbar=ttk.Progressbar(progress_screen,orient="horizontal",length=700,mode="determinate")
    progressbar.place(x=400,y=400)
    progressbar.configure(maximum=X*Y,variable=pbval)
    #label
    progress_label1 = tk.Label(progress_screen,text="generating images", background='#505050',  foreground='#ccffff',font=("Lucida Console","15"))
    progress_label1.place(x=400, y=375)
    progress_label2 = tk.Label(progress_screen,text="please wait...", background='#505050', foreground='red',font=("Lucida Console","10"))
    progress_label2.place(x=1000, y=425)
    root.update()
    ##############################################
    #画像を生成する
    print(type+": images are generating")
    diffusion_images = []
    if type=="iec":
        for i in range(population):
            diffusion_image = func(genes[i][0],genes[i][1])
            diffusion_images.append(diffusion_image)
            pbval.set(pbval.get() + 1)
            root.update()
        save_image(diffusion_images,iec_epoch,iec_image_save_path)
    elif type=="conv":
        global prompt
        global conv_epoch
        prompt = []
        input_prompt = user_input_prompt.get(1.0, tk.END+"-1c")
        # ログ
        with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
            print(time.time(), "conv_epoch", conv_epoch, "input_prompt",input_prompt, file=f)
        text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
        with torch.no_grad():
            text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
        diffusion_images = []
        for i in range(population):
            seed_here = random.randrange(1000)
            latents_torch = torch.randn(
              (batch_size, unet.in_channels, height // 8, width // 8),
              generator=torch.manual_seed(seed_here),
            )
            latents = latents_torch.to(torch_device)
            diffusion_image = func(latents,text_embeddings)
            diffusion_images.append(diffusion_image)
            pbval.set(pbval.get() + 1)
            root.update()
        save_image(diffusion_images,conv_epoch,stable_diffusion_image_save_path)
    elif type=="px1" or type=="px2":
        for i in range(population):
            time.sleep(1.0/population)
            pbval.set(pbval.get() + 1)
            root.update()
    else:
        print('Error: progress screen invalid input', file=sys.stderr)
        sys.exit(1)
    progress_screen.destroy() #prompt入力画面を削除する
# web表示用の関数
# 選択されたときにボタンの色を変えて、選ばれた画像を記録
def click_button(item,btn):
    def nothing():
        if iec_selected[item]==1:
            btn.config(bg='#E6E6E6')
            iec_selected[item] = 0
        else:
            btn.config(bg='yellow')
            iec_selected[item] = 1
    return nothing
# evolveボタンが押されたときに進化させて新しい画像を表示
def click_evolve_button():
    global iec_epoch
    #################
    # stable diffusionのコード
    #進化+選択
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_epoch, "selected population", iec_selected, file=f)
    #進化
    global genes
    genes = evolve(iec_selected,genes)
    #画像生成
    iec_epoch += 1
    if current_system_is_live:
        showing_progress_screen_and_generate_images("iec",evolve_screen)
    elif current_system_is_practice1:
        showing_progress_screen_and_generate_images("px1",evolve_screen)
    else:
        print('Error: evolve system error', file=sys.stderr)
        sys.exit(1)
    ###################
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_epoch, "new images are displayed", file=f)
    #新しい画像を表示
    iec_image_path = "sample_image"
    for i in range(X*Y):
        if current_system_is_live:
            iec_image_path = iec_image_save_path+'/epoch'+str(iec_epoch)+'/' + str(i) + '.png'
        elif current_system_is_practice1:
            iec_image_path = sample_image_folder_path+'/epoch'+str(iec_epoch)+'/' +str(i) + '.png'
        img = Image.open(iec_image_path)
        img = img.resize((256,256),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        imgs.append(img) # Imageをリストに追加
        canvases[i].delete()
        canvases[i].create_image(3, 3, image=img, anchor=tk.NW,tag=str(i))
        #ボタンを未クリックにする
        iec_buttons[i].config(bg='#E6E6E6')
        iec_selected[i] = 0
    labels[0].config(text=evolve_explanation)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_epoch, "evolve_explanation",evolve_explanation, file=f)
#最後の結果画面
def display_final_result_screen():
    experiment_end_screen.destroy()
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "final_result_screen_is_displayed.", file=f)
    # iec target画像の設置
    set_target_title("iec",root)
    # iec generated画像の設置
    img = Image.open(iec_image_save_path+'/epoch'+str(iec_epoch)+'/' + str(iec_selected_image_id) + '.png')
    img = img.resize((256,256),Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    imgs.append(img) # Imageをリストに追加
    canvas = tk.Canvas(root, width=256, height=256)
    canvas.place(x=500, y=100) 
    canvas.create_image(3, 3, image=img, anchor=tk.NW)
    iec_generated_label = tk.Label(root,text="image 1", foreground='blue',font=("Lucida Console","15"))
    iec_generated_label.place(x=500, y=70)
    # stable diffusion target labelの設置
    target_label = tk.Label(root,text="theme", foreground='blue',font=("Lucida Console","15"))
    target_label.place(x=150, y=470)
    target_theme_label = tk.Label(root,text=conv_target_theme, foreground='black',font=("Lucida Console","15"))
    target_theme_label.place(x=150, y=500)
    # stable diffusion generated画像の設置
    img = Image.open(stable_diffusion_image_save_path+'/epoch'+str(conv_epoch)+'/' + str(conv_selected_image_id) + '.png')
    img = img.resize((256,256),Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    imgs.append(img) # Imageをリストに追加
    canvas = tk.Canvas(root,width=256, height=256)
    canvas.place(x=500, y=500) 
    canvas.create_image(3, 3, image=img, anchor=tk.NW)
    iec_generated_label = tk.Label(root,text="image 2", foreground='blue',font=("Lucida Console","15"))
    iec_generated_label.place(x=500, y=470)
#実験終了画面
def display_experiment_end_screen():
    if reverse_experiment_order.get()==False:
        #iec->conv
        global conv_selected_image_id
        #画像を未選択だったときにエラー処理
        conv_selected_image_id = conv_now_selected_image_id.get()
        if conv_selected_image_id==population:
            messagebox.showerror('Error', 'Please select one image before you press the end button')
            return
        # 実験済みにチェックをつける
        global conv_experiment_done
        conv_experiment_done=True
        #進化計算画面を削除する
        conv_choosing_screen.destroy()
    else:
        #conv->iec
        global iec_selected_image_id
        #画像を未選択だったときにエラー処理
        iec_selected_image_id = iec_now_selected_image_id.get()
        if iec_selected_image_id==population:
            messagebox.showerror('Error', 'Please select one image before you press the end button')
            return
        # 実験済みにチェックをつける
        global iec_experiment_done
        iec_experiment_done=True
        #進化計算画面を削除する
        evolve_screen.destroy()
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "experiment_end_screen_is_displayed", file=f)
    messagebox.showinfo('good job!', 'This is the end of this program')
    #実験終了画面でのframe作成
    global experiment_end_screen
    experiment_end_screen = tk.Frame(root,width=1500,height=900)
    experiment_end_screen.pack(fill=tk.BOTH)
    #スコア表示ボタンボタン
    display_result_button = tk.Button(experiment_end_screen,text = "result", fg="white", bg="blue",command=display_final_result_screen)
    display_result_button.place(x=690,y=440)
#stable diffusionでの画像生成
def click_generate_button():
    global conv_epoch
    conv_epoch += 1
    #ui上に画像を表示する
    #stable diffusionで画像を生成する
    if current_system_is_live:
        showing_progress_screen_and_generate_images("conv",conv_choosing_screen)
    elif current_system_is_practice2:
        showing_progress_screen_and_generate_images("px2",conv_choosing_screen)
    else:
        print('Error: system changing error', file=sys.stderr)
        sys.exit(1)
    #第2世代以降の画像を表示する
    conv_image_path = "sample image"
    for i in range(X*Y):
        if current_system_is_live:
            conv_image_path = stable_diffusion_image_save_path+'/epoch'+str(conv_epoch)+'/' + str(i) + '.png'
        elif current_system_is_practice2:
            conv_image_path = sample_image_folder_path + '/epoch'+str(conv_epoch)+'/' + str(i) + '.png'
        img = Image.open(conv_image_path)
        img = img.resize((256,256),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        imgs.append(img) # Imageをリストに追加
        conv_canvases[i].delete()
        conv_canvases[i].create_image(3, 3, image=img, anchor=tk.NW,tag=str(i))
#従来手法で実験を行う
def display_conv_choosing_screen():
    global user_input_prompt
    input_prompt=user_input_prompt.get(1.0, tk.END+"-1c")#frame_destroyの前に取得する必要がある
    #stable diffusionで画像を生成する
    if current_system_is_live:
        showing_progress_screen_and_generate_images("conv",first_prompt_input_screen)
    elif current_system_is_practice2:
        showing_progress_screen_and_generate_images("px2",first_prompt_input_screen)
    else:
        print('Error: system changing error conv', file=sys.stderr)
        sys.exit(1)
    #first prompt入力画面を削除する
    first_prompt_input_screen.destroy()
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "conv_choosing_screen_is_displayed.", file=f)
    #従来手法画像生成画面でのframe作成
    global conv_choosing_screen
    conv_choosing_screen = tk.Frame(root,width=1500,height=900)
    conv_choosing_screen.pack(fill=tk.BOTH)
    #初期世代の画像を表示する
    conv_image_path = "iec_sample_image"
    for i in range(X):
        for j in range(Y):
            # 選択肢の画像の設置
            if current_system_is_live:
                conv_image_path = stable_diffusion_image_save_path+'/epoch0/' + str(i*Y+j) + '.png'
            elif current_system_is_practice2:
                conv_image_path = sample_image_folder_path + '/epoch0/' + str(i*Y+j) + '.png'
            img = Image.open(conv_image_path)
            img = img.resize((256,256),Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            imgs.append(img)
            canvas = tk.Canvas(conv_choosing_screen, width=256, height=256)
            canvas.place(x=(j+2)*image_padding, y=i*image_padding) 
            canvas.create_image(3, 3, image=img, anchor=tk.NW,tag=str(i*Y+j))
            conv_canvases.append(canvas)
            # 選んでもらうボタンの設置
            image_id_label = tk.Label(conv_choosing_screen, text=str(i*Y+j), width=5, height=2,bg="white")
            image_id_label.place(x=(j+2.35)*image_padding,y=(i+0.85)*image_padding)
    #target画像の表示
    set_target_title("conv",conv_choosing_screen)
    #prompt入力ボックスの設置
    conv_prompt_frame = tk.Frame(conv_choosing_screen, bg="#C6C6C6",width=256,height=300)
    conv_prompt_frame.place(x=150,y=500)
    # promptの入力画面を作成
    conv_screen_explanation_frame = ttk.Label(conv_prompt_frame, text="input the description of \nthe image")
    conv_screen_explanation_frame.pack(padx=5, pady=10, fill=tk.X)
    user_input_prompt = tk.Text(conv_prompt_frame, height=10)
    user_input_prompt.pack(padx=5, pady=10, fill=tk.X)
    user_input_prompt.insert(1.0, input_prompt)
    # generateボタン
    conv_screen_generate_image_button = tk.Button(conv_prompt_frame,fg="white", bg="blue", text="generate", command=click_generate_button)
    conv_screen_generate_image_button.place(x=80,y=170)
    # 気に入った唯一の画像を選ぶボタン
    number_select = list(range(population+1))
    global conv_now_selected_image_id
    conv_now_selected_image_id = tk.IntVar()
    number_select_pulldown = ttk.Combobox(
        conv_prompt_frame, textvariable=conv_now_selected_image_id, state="readonly",
        values=number_select, width=10)
    number_select_pulldown.set(population)
    number_select_pulldown.place(x=40, y=275)
    # end ボタン
    conv_screen_fix_image_button = tk.Button(conv_prompt_frame,bg="red", text="end")
    conv_screen_fix_image_button.place(x=150,y=275)
    if current_system_is_practice2:
        conv_screen_fix_image_button.config(command=display_start_screen)
    elif reverse_experiment_order.get()==False:
        #iec->conv
        conv_screen_fix_image_button.config(command=display_experiment_end_screen)
    else:
        #conv->iec
        conv_screen_fix_image_button.config(command=display_going_next_screen)
    conv_choosing_screen.propagate(False)
    conv_prompt_frame.propagate(False)
##############################################
# １つ目の実験が終わったときに次の実験に進むボタンの表示
def display_going_next_screen():
    if reverse_experiment_order.get()==False:
        #iec->conv
        global iec_selected_image_id
        #画像を未選択だったときにエラー処理
        iec_selected_image_id = iec_now_selected_image_id.get()
        if iec_selected_image_id==population:
            messagebox.showerror('Error', 'Please select one image before you press the end button')
            return
        # 実験済みにチェックをつける
        global iec_experiment_done
        iec_experiment_done=True
        #進化計算画面を削除する
        evolve_screen.destroy()
    else:
        #conv->iec
        global conv_selected_image_id
        #画像を未選択だったときにエラー処理
        conv_selected_image_id = conv_now_selected_image_id.get()
        if conv_selected_image_id==population:
            messagebox.showerror('Error', 'Please select one image before you press the end button')
            return
        # 実験済みにチェックをつける
        global conv_experiment_done
        conv_experiment_done=True
        #進化計算画面を削除する
        conv_choosing_screen.destroy()
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "going_next_screen_is_displayed", file=f)
    #次の実験に進む画面でのframe作成
    global going_next_screen
    going_next_screen = tk.Frame(root,width=1500,height=900)
    going_next_screen.pack(fill=tk.BOTH)
    #次の実験に進むボタン
    going_next_button = tk.Button(going_next_screen,text = "next experiment", fg="white", bg="blue",command=display_first_prompt_input_screen)
    going_next_button.place(x=690,y=440)
# iec画面の作成
def display_iec_choosing_screen():
    input_prompt = user_input_prompt.get(1.0, tk.END+"-1c")#frame_destroyの前に取得する必要がある
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_input_prompt", input_prompt, file=f)
    # stable diffusionのコード
    text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
      text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    global genes
    genes = initialize_gene(text_embeddings)
    first_prompt_input_screen.destroy() #prompt入力画面を削除する
    if current_system_is_live:
        showing_progress_screen_and_generate_images("iec",root)
    elif current_system_is_practice1:
        showing_progress_screen_and_generate_images("px1",root)
    else:
        print('Error: system changing error', file=sys.stderr)
        sys.exit(1)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "display_iec_choosing_screen_is_displayed.", file=f)
    #進化計算画面でのフレームの作成
    global evolve_screen
    evolve_screen = tk.Frame(root,width=1500,height=900)
    evolve_screen.pack(fill=tk.BOTH)
    #初期世代の画像を表示する
    iec_image_path = "iec_sample_image"
    for i in range(X):
        for j in range(Y):
            # 選択肢の画像の設置
            if current_system_is_live:
                iec_image_path = iec_image_save_path+'/epoch0/' + str(i*Y+j) + '.png'
            elif current_system_is_practice1:
                iec_image_path = sample_image_folder_path + '/epoch0/' + str(i*Y+j) + '.png'
            img = Image.open(iec_image_path)
            img = img.resize((256,256),Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            imgs.append(img)
            canvas = tk.Canvas(evolve_screen, width=256, height=256)
            canvas.place(x=(j+2)*image_padding, y=i*image_padding) 
            canvas.create_image(3, 3, image=img, anchor=tk.NW,tag=str(i*Y+j))
            canvases.append(canvas)
            # 選んでもらうボタンの設置
            button = tk.Button(evolve_screen, text=str(i*Y+j),width=3, height=1, bg="white")
            button.place(x=(j+2.35)*image_padding,y=(i+0.85)*image_padding)
            button.config(command=click_button(i*Y+j,button))
            iec_buttons.append(button)
    evolve_explanation = input_prompt #説明文
    evolution_label = tk.Message(evolve_screen,text=evolve_explanation,  foreground='black',font=("Lucida Console","15"), width=256)
    evolution_label.place(x=150, y=400)
    labels.append(evolution_label)
    # ボタン用のフレーム
    iec_navigate_frame = tk.Frame(evolve_screen, bg="#C6C6C6",width=256,height=100)
    iec_navigate_frame.place(x=150,y=500)
    #進化させるボタン
    evolve_button = tk.Button(iec_navigate_frame,text = "continue", fg="white", bg="blue",command=click_evolve_button)
    evolve_button.place(x=85,y=10)
    # 気に入った唯一の画像を選ぶボタン
    global iec_now_selected_image_id
    iec_now_selected_image_id = tk.IntVar()
    iec_number_select_pulldown = ttk.Combobox(
        iec_navigate_frame, textvariable=iec_now_selected_image_id, state="readonly", 
        values=list(range(population+1)), width=10)
    iec_number_select_pulldown.set(population)
    iec_number_select_pulldown.place(x=40, y=75)
    # 終了ボタン
    end_button = tk.Button(iec_navigate_frame,text = "end", bg="red")
    end_button.place(x=150,y=75)
    
    if current_system_is_practice1:
        end_button.config(command=display_start_screen)
    elif reverse_experiment_order.get()==False:
        #iec->conv
        end_button.config(command=display_going_next_screen)
    else:
        #conv->iec
        end_button.config(command=display_experiment_end_screen)
    # target画像の設置
    set_target_title("iec",evolve_screen)
#promptを初めて入力するときの入力画面の作成
def display_first_prompt_input_screen():
    if iec_experiment_done==False and conv_experiment_done==False:
        start_screen.destroy()
    elif iec_experiment_done==False or conv_experiment_done==False:
        going_next_screen.destroy()
    else:
        print('Error: screen switching error', file=sys.stderr)
        sys.exit(1)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "first_prompt_input_screen_is_displayed.", file=f)
    global first_prompt_input_screen
    first_prompt_input_screen = tk.Frame(root,width=1500,height=900)
    first_prompt_input_screen.pack(fill=tk.BOTH)
    prompt_frame = tk.Frame(first_prompt_input_screen,width=400,height=350)
    prompt_frame.place(x=650,y=50)
    # promptの入力画面を作成
    first_prompt_input_screen_explanation_frame = tk.Label(prompt_frame, bg='#C6C6C6',text="input the description of the image")
    first_prompt_input_screen_explanation_frame.pack(padx=5, pady=10, fill=tk.X)
    global user_input_prompt
    user_input_prompt = tk.Text(prompt_frame, height=10,font=("Ariel","15"))
    user_input_prompt.pack(padx=5, pady=20, fill=tk.X)
    #generateボタン
    first_prompt_input_screen_iec_generate_image_button = tk.Button(prompt_frame,fg="#FFFFFF", 
                               width=5,height=6,bg='#1E88E5',text="決定\ngenerate images",font=('gothic',"30"))
    first_prompt_input_screen_iec_generate_image_button.pack(padx=5, pady=10, fill=tk.X)
    if current_system_is_practice1 or (current_system_is_live and iec_experiment_done==False and conv_experiment_done==False and reverse_experiment_order.get()==False)or (iec_experiment_done==False and conv_experiment_done and reverse_experiment_order.get()):
        # generateを押すとiec画像生成
        first_prompt_input_screen_iec_generate_image_button.config(command=display_iec_choosing_screen)
        # お題の設置
        set_target_title("iec",first_prompt_input_screen)
    elif current_system_is_practice2 or (iec_experiment_done and conv_experiment_done==False and reverse_experiment_order.get()==False) or (iec_experiment_done==False and conv_experiment_done==False and reverse_experiment_order.get()):
        # generateを押すとconv画像生成
        first_prompt_input_screen_iec_generate_image_button.config(command=display_conv_choosing_screen)
        # お題の設置
        set_target_title("conv",first_prompt_input_screen)
    else:
        print('Error: screen switching error', file=sys.stderr)
        sys.exit(1)
    first_prompt_input_screen.propagate(False)
    prompt_frame.propagate(False)
    # 写真のサンプルを提示
    showing_sample_screen = tk.Frame(first_prompt_input_screen,bg='#C6C6C6',width=1500,height=450)
    showing_sample_screen.place(x=0,y=450)
    img = Image.open(sample_image_path)
    img = img.resize((256,256),Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    imgs.append(img) # Imageをリストに追加
    canvas = tk.Canvas(first_prompt_input_screen, width=256, height=256, bg='#C6C6C6')
    canvas.place(x=150, y=500) 
    canvas.create_image(3, 3, image=img, anchor=tk.NW,tag="sample image")
    target_label = tk.Label(first_prompt_input_screen,text="sample image", background='#C6C6C6',foreground='blue',font=("Lucida Console","15"))
    target_label.place(x=150, y=470)
    # promptのサンプルを提示
    sample_prompt_frame = tk.Frame(first_prompt_input_screen,width=400,height=256)
    sample_prompt_frame.place(x=650,y=500)
    sample_prompt_explanation_label = ttk.Label(sample_prompt_frame, text="This is an example of the description")
    sample_prompt_explanation_label.pack(padx=5, pady=10, fill=tk.X)
    sample_prompt_label = tk.Message(sample_prompt_frame, bg='#C6C6C6',text=sample_image_prompt,width=380)
    sample_prompt_label.pack(padx=5, pady=10, fill=tk.X)
    sample_prompt_frame.propagate(False)
# 練習1に飛んだら練習1のグローバル変数をTrueにする
def switch_to_practice1():
    iec_image_save_path='./user_experiment_data/practice/iec'
    stable_diffusion_image_save_path='./user_experiment_data/practice/stable_diffusion'
    global experiment_id
    experiment_id = "practice"
    #ディレクトリ初期化
    if os.path.exists('./user_experiment_data/practice'):
        shutil.rmtree('./user_experiment_data/practice')
    os.mkdir('./user_experiment_data/practice')
    os.mkdir(iec_image_save_path)
    os.mkdir(stable_diffusion_image_save_path)
    global current_system_is_practice1,current_system_is_practice2,current_system_is_live
    current_system_is_practice1 = True
    current_system_is_practice2 = False
    current_system_is_live = False
    display_first_prompt_input_screen()
# 練習2に飛んだら練習2のグローバル変数をTrueにする
def switch_to_practice2():
    iec_image_save_path='./user_experiment_data/practice/iec'
    stable_diffusion_image_save_path='./user_experiment_data/practice/stable_diffusion'
    global experiment_id
    experiment_id = "practice"
    #ディレクトリ初期化
    if os.path.exists('./user_experiment_data/practice'):
        shutil.rmtree('./user_experiment_data/practice')
    os.mkdir('./user_experiment_data/practice')
    os.mkdir(iec_image_save_path)
    os.mkdir(stable_diffusion_image_save_path)
    global current_system_is_practice1,current_system_is_practice2,current_system_is_live
    current_system_is_practice1 = False
    current_system_is_practice2 = True
    current_system_is_live = False
    display_first_prompt_input_screen()
# 本番に飛んだら本番のグローバル変数をTrueにする
def switch_to_live():
    #被験者のidを振る
    experiment_time_delta = datetime.timedelta(hours=9)
    experiment_JST = datetime.timezone(experiment_time_delta, 'JST')
    global experiment_id,iec_image_save_path,stable_diffusion_image_save_path
    experiment_now = datetime.datetime.now(experiment_JST)
    experiment_id = experiment_now.strftime('%Y%m%d%H%M%S')
    iec_image_save_path='./user_experiment_data/'+experiment_id+'/iec'
    stable_diffusion_image_save_path='./user_experiment_data/'+experiment_id+'/stable_diffusion'
    #ディレクトリ初期化
    os.mkdir('./user_experiment_data/'+experiment_id)
    os.mkdir(iec_image_save_path)
    os.mkdir(stable_diffusion_image_save_path)
    global current_system_is_practice1,current_system_is_practice2,current_system_is_live
    current_system_is_practice1 = False
    current_system_is_practice2 = False
    current_system_is_live =True
    display_first_prompt_input_screen()
def display_start_screen():
    if current_system_is_practice1:
        evolve_screen.destroy()
    elif current_system_is_practice2:
        conv_choosing_screen.destroy()
    global start_screen
    #webの初期配置コード
    start_screen = tk.Frame(root,width=1500,height=900)
    start_screen.pack(fill=tk.BOTH)
    #練習1 ボタン
    start_screen_practice1_button = tk.Button(start_screen,text="練習1\npractice 1", command=switch_to_practice1,font=("gothic","50"))
    start_screen_practice1_button.config(bg="#1E88E5", fg='#FFFFFF', width=20, height=3)
    start_screen_practice1_button.place(x=450,y=100)
    #練習2 ボタン
    start_screen_practice2_button = tk.Button(start_screen,text="練習2\npractice 2", command=switch_to_practice2,font=("gothic","50"))
    start_screen_practice2_button.config(bg="#42A5F5", fg='#FFFFFF', width=20, height=3)
    start_screen_practice2_button.place(x=450,y=350)
    #本番 ボタン
    start_screen_live_button = tk.Button(start_screen,text="本番\nlive", command=switch_to_live,font=("gothic","50"))
    start_screen_live_button.config(bg="#E53935", fg='#FFFFFF', width=20, height=3)
    start_screen_live_button.place(x=450,y=600)
    #各種パラメーター調整
    #iecとstable diffusionの順序
    global reverse_experiment_order
    reverse_experiment_order = tk.BooleanVar()
    experiment_order_checkbox = tk.Checkbutton(start_screen,variable=reverse_experiment_order, text='switch')
    experiment_order_checkbox.place(x=50, y=850)
if __name__ == "__main__":
    with autocast("cuda"):
        ############################################################################
        #webの初期配置(ptompt入力画面)
        root = tk.Tk()
        root.title("web browser")
        root.geometry("1500x900")
        display_start_screen()
        root.mainloop()